<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/Language_Model_Text_Generation_in_Pytorch_using_RNN_with_(LSTM)_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount("/content/drive/", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>You are on Google CoLaB with Pytorch Version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>Please correct {type(e)} and reload your device")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t: float = 123.781)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>time formating\tplease wait...\n>>>time elapsed\t {time_fmt()}")

Mounted at /content/drive/
>>>You are on Google CoLaB with Pytorch Version: 1.8.1+cu101
>>>time formating	please wait...
>>>time elapsed	 0: 02: 03.00


In [26]:
#In this notebook we are going to train a character level language model to generate new texts
#For demonstration we will apply a dataset with bunch of names to generate new names (Any text can be applied)
#We are going to use rnn with an LSTM architecture 

In [27]:
!pip install 'unidecode'

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.


In [28]:
import torch, string
import torch.nn as nn
import torch.optim as optim
import time, sys,string, random
import unidecode, os
from torch.utils.tensorboard import SummaryWriter

In [29]:
#We first read-in the text file which contain our data for this project:
os.chdir("/content/drive/MyDrive/namesGen")
data_file = unidecode.unidecode(open(file = 'Names.txt').read())

In [30]:
#print few lines of data.
print(data_file[:297])


Aaban
Aabharan
Aabhas
Aabhat
Aabheer
Aabheer
Abheer
Aabher
Aabi
Aabilesh
Aabir
Aabishan
Aabishayan
Aacharya
Aachman
Aachuthan
Aadalalagan
Aadalarasan
Aadalazhagan
Aadamya
Aadanyan
Aadarko
Aadarsh
Aadarshan
Aadarshanan
Aadav
Aadavan
Aadesh
Aadesh
Adesh
Aadhan
Aadhar
Aadhav
Aadhavan
Aadhi
Aadhiban



In [31]:
#Fetch reference characters from strings.prentable
chars = string.printable
num_chars = len(chars)

In [32]:
print(f"The reference characters are : {chars}\nThe size of ref_char: {len(chars)}")

The reference characters are : 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

The size of ref_char: 100


In [41]:
#We start by building our LSTM-class (The ussual LSTM):
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        out = self.embed(x)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden, cell)

    def _initializer_(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell
  


In [42]:
#We now build our generator class:
class TextGen(nn.Module):
  def __init__(self):
    super(TextGen, self).__init__()
    self.learning_rate = 1e-3
    self.batch_size = 1
    self.num_epochs = 2000
    self.every_epoch = 50
    self.text_chunk = 250
    self.num_layers = 2
    self.hidden_size = 256
  
  def _get_indices(self, string):
    '''This method build a tensor of indices from string'''
    #place holder for the indeces
    tensor = torch.zeros(len(string)).long()
    #Fetch the index for every char from the reference list and store in tensor
    for c in range(len(string)):
      tensor[c] = chars.index(string[c])
    return tensor
  
  def _rndbatch_(self):
    '''select random chunk of texts from the data file'''
    start_idx = random.randint(0, len(data_file) - self.text_chunk)
    end_idx = start_idx + self.text_chunk + 1
    text_str = data_file[start_idx:end_idx]
    #place holder for the input and output data
    input_text = torch.zeros(self.batch_size, self.text_chunk)
    target_text = torch.zeros(self.batch_size, self.text_chunk)
    for i in range(self.batch_size):
      input_text[i,:] = self._get_indices(text_str[:-1])
      target_text[i,:] = self._get_indices(text_str[1:]) #We offset by 1 to predict next char
    return input_text.long(), target_text.long()
  
  def _generate_(self, initial_str = 'Aa', pred_len = 100, temperature = 0.85):
    h0,c0 = self.rnn._initializer_(batch_size = self.batch_size)
    initial_input = self._get_indices(initial_str)
    predicted = initial_str #first string always predicted to the same
    for p in range(len(initial_str) - 1):
      _, (hidden,cell) = self.rnn(
          initial_input[p].view(1).to(device), c0,h0)
    last_char = initial_input[-1]

    for p in range(pred_len):
      out, (hidden, cell) = self.rnn(last_char.view(1).to(device), h0, c0)
      out_dstn = out.data.view(-1).div(temperature).exp()
      top_chars = torch.multinomial(out_dstn,1)[0]
      predicted_char = chars[top_chars]
      predicted+=predicted_char
      last_char = self._get_indices(predicted_char)
    return predicted
  
  #Now we trin our model (through the following train function)
  def train(self):
    self.rnn = LSTM(num_chars, self.hidden_size, self.num_layers,num_chars).to(device)
    optimizer = optim.Adam(params = self.rnn.parameters(), lr = self.learning_rate)
    loss_obj = nn.CrossEntropyLoss()
    writer = SummaryWriter(f"runs/names0")
    print(f"\n>>>training starts please wait....")
    for epoch in range(1, self.num_epochs + 1):
      data, target = self._rndbatch_()
      h0,c0 = self.rnn._initializer_(self.batch_size)

      self.rnn.zero_grad()
      loss = 0
      data = data.to(device)
      target = target.to(device)
      for c in range(self.text_chunk):
        output, (hidden, cell) = self.rnn(data[:,c], h0,c0)
        loss+=loss_obj(output, target[:,c])
      
      loss.backward()
      optimizer.step()
      loss = loss.item()/self.text_chunk

      if epoch % self.every_epoch == 0:
        print(f"\n>>>>Loss is: {loss:.4f}")
        print(f"\n>>>>{self._generate_()}")
      writer.add_scalar("Training loss", loss, global_step = epoch)
  

In [43]:
#Running the model to generate new names
names_gen = TextGen()

In [44]:
tic = time.time()
names_gen.train()
toc = time.time()
print(f"\n>>>>time elapsed: {time_fmt(toc - tic)}")


>>>training starts please wait....

>>>>Loss is: 2.8447

>>>>Aan
enyi
QAjee
Kanakarirayajalva
jalinteujavavai
thathan
+Q`
rasha
ffdhi
ha
6|z9Rath
0nran
Sharal
Vimh

>>>>Loss is: 2.4105

>>>>Aabukurinubar
Atha
Knryananminthashava
Har
Tha
Dha
Sajidran
Anar
Raonimethasotrathathaeyurilirajishava

>>>>Loss is: 2.2956

>>>>Aa
Kener
Vearikaleei
Su
Dama
Keerajorannanineakarajana
Kanerathinthoparagadea
Mera
Ran
Ke
Mhathath
Ya


>>>>Loss is: 2.6865

>>>>Aamanavishanith
Vo
Kininugeysho+niI]|5anarishitashajothrin
Sajalrinanta
Jithrithari
Noraruyaveshn
Kish

>>>>Loss is: 2.4908

>>>>Aakubil
Naleygan
Han
UhukanevoPakalarApin
Danubhraminthi
Dhaganunavaranan
Abharthan
Dham
Gartukuravi
A

>>>>Loss is: 2.9119

>>>>Aarilandrekash
Paan
Jeshdhra
An
Panvin
Van
Vthan
Pa
Bakayalthatathljmalanal
Jan
Ashareshithanythimir
D

>>>>Loss is: 2.3865

>>>>Aani
Asha
Mangashikathan
So
Rasrnuta
Vizigema
Thiyanashin
Avara
Handhakedhika
Ni
T
Vra
Ptha
Ra
Kaana
S

>>>>Loss is: 2.2878

>>>>Aar
Khit
banibhanganandashanyali
Vi